In [1]:
from qiskit import QuantumRegister, ClassicalRegister
from qiskit import QuantumCircuit, compile, IBMQ
IBMQ.load_accounts()

Let's look at the coupling map of melbourne and find five qubits in a line.

In [2]:
backend=IBMQ.get_backend('ibmq_16_melbourne')
backend.configuration().coupling_map

[[1, 0],
 [1, 2],
 [2, 3],
 [4, 3],
 [4, 10],
 [5, 4],
 [5, 6],
 [5, 9],
 [6, 8],
 [7, 8],
 [9, 8],
 [9, 10],
 [11, 3],
 [11, 10],
 [11, 12],
 [12, 2],
 [13, 1],
 [13, 12]]

Looks like 0-1-2-3-4 will work. Now we make a circuit using only two qubit gates between neighbouring pairs on this line.

In [3]:
q = QuantumRegister(14)
c = ClassicalRegister(5)
qc = QuantumCircuit(q, c)

qc.cx(q[0],q[1])
qc.cx(q[2],q[3])
qc.cx(q[2],q[1])
qc.cx(q[4],q[3])

for j in range(5):
    qc.measure(q[j],c[j])

qobj = compile(qc,backend)

There was no need for the compiler to add any addition cnots. So let's see if the compiled circuit only has the four cnots we want.

In [4]:
qobj.as_dict()['experiments'][0]['header']['compiled_circuit_qasm'].count('cx')

10

It has more. But why?